In [1]:
import json

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse each line as a JSON object
            data.append(json.loads(line))
    return data

# Usage
file_path = 'output.jsonl'  # Replace with the path to your JSONL file
contents = load_jsonl(file_path)
print(len(contents))

text_contents = [slide['content'] for slide in contents]
print(len(text_contents))
print(type(text_contents))
print(text_contents[0])


90
90
<class 'list'>
La diapositive présente les informations suivantes :

- Il s'agit de la première semaine ("Semaine 1") d'un cours, qui est dédié à la présentation du cours en question.
- Le titre du cours est "INF889G - Vision par ordinateur".
- Le nom de l'intervenant ou du responsable du cours est "Joël Lefebvre".
- L'institution affiliée au cours est l'UQAM, qui pourrait représenter l'Université du Québec à Montréal.
- Le cours se déroule durant la session "Hiver 2024".

Résumé : La diapositive introduit la première semaine d'un cours de vision par ordinateur, intitulé INF889G, dirigé par Joël Lefebvre à l'UQAM pour la session de l'hiver 2024.


In [2]:
import os
import cohere
import numpy as np
from dotenv import load_dotenv

load_dotenv()

model_name = "embed-multilingual-v3.0"
api_key: str | None = os.getenv("COHERE_API_KEY")
input_type_embed = "search_document"

# Now we'll set up the cohere client.
if api_key is None:
    raise ValueError("Please set the COHERE_API_KEY environment variable.")
co = cohere.Client(api_key)

# Get the embeddings
embeds: list[list[float]] = co.embed(
    texts=text_contents, model=model_name, input_type=input_type_embed
).embeddings

# print(embeds)
print(type(embeds), len(embeds), len(embeds[0]))
array_embeds = np.array(embeds)
print(array_embeds.shape)
np.save('embeddings.npy', array_embeds)

<class 'list'> 90 1024
(90, 1024)


In [3]:
import numpy as np

def cosine_similarity_matrix(vectors, query_vec):
    dot_product = np.dot(vectors, query_vec)
    
    norms_vectors = np.linalg.norm(vectors, axis=1)
    norm_query_vec = np.linalg.norm(query_vec)
    
    if norm_query_vec == 0 or np.any(norms_vectors == 0):
        raise ValueError("Cosine similarity is not defined when one or both vectors are zero vectors.")
    
    similarity = dot_product / (norms_vectors * norm_query_vec)
    return similarity


In [4]:
import os
import cohere
from dotenv import load_dotenv

load_dotenv()

model_name = "embed-multilingual-v3.0"
api_key: str | None = os.getenv("COHERE_API_KEY")
input_type_embed = "search_query"

QUERY="Le devoir 1 vaut pour combien de pourcentage?"

# Now we'll set up the cohere client.
if api_key is None:
    raise ValueError("Please set the COHERE_API_KEY environment variable.")
co = cohere.Client(api_key)

# Get the embeddings
query_embed: list[list[float]] = co.embed(
    texts=[QUERY], model=model_name, input_type=input_type_embed
).embeddings
query_array = np.array(query_embed)
query_array = query_array.reshape(-1)

embeds_dataset = np.load('embeddings.npy')

similarity_results = np.zeros((embeds_dataset.shape[0],), dtype=np.float32)
try:
    similarity_results = cosine_similarity_matrix(embeds_dataset, query_array)
except ValueError as e:
    print(e)

sorted_indices = np.argsort(similarity_results)[::-1]
relevant_contents = [contents[i] for i in sorted_indices[:5]]
for i in range(8):
    print(similarity_results[sorted_indices[i]], contents[sorted_indices[i]])

# -----------------------------------------------
from openai import OpenAI
def extract_tags_function_call(input_prompt: str, model: str, max_retries: int = 1):
    client = OpenAI()
    try:
        extracted_details = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a world class teacher in the field of management and supervision of teams working on projects. You will answer questions asked by students. You will use theory provided to you in the form of slides.",
                },
                {
                    "role": "user",
                    "content": input_prompt,
                },
            ],
            max_tokens=1000,
        )
        return extracted_details

    except Exception as e:
        print(e)
        return f"Error generating post with the OpenAI API: {e}"
    
input_prompt = f"REMEMBER TO USE THE THEORY GIVEN TO YOU. But not all of it might be relevant to the question. Here is the student's question: {QUERY}\n, and the relevant theory: {relevant_contents}. Give a clear answer that helps the student understand the course material. Cite the sources you used to answer the question."
answer = extract_tags_function_call(input_prompt, "gpt-3.5-turbo-1106")
print('\n')
print(answer.choices[0].message.content)

0.5705430007241215 {'module': 'data/images', 'slide_number': 'page_10.png', 'content': 'La diapositive présente les "Modalités d\'évaluation" pour un cours, indiquant les pourcentages attribués à chaque élément d\'évaluation ainsi que leurs échéances respectives. Les évaluations sont réparties comme suit :\n\n- Devoir 1 : 15% de la note totale, pour la semaine 4.\n- Devoir 2 : 15% de la note totale, pour la semaine 7.\n- Examen intra : 25% de la note totale, pour la semaine 9.\n- Devoir 3 : 15% de la note totale, pour la semaine 13.\n- Projet : 30% de la note totale, pour la semaine 17.\n\nLe projet est décrit comme étant divisé en trois parties et des détails supplémentaires sont à venir. Les trois parties sont :\n\n1. Proposition de projet : 5%, semaine 9.\n2. Rapport : 15%, semaine 17.\n3. Présentation : 10%, semaine 17.\n\nEn résumé, la diapositive fournit les pourcentages et les délais pour chaque composant d\'évaluation d\'un cours, et détaille la structure de l\'évaluation du pr